<a href="https://colab.research.google.com/github/invictus125/cs598-final-project/blob/main/intraoperative_hypotension.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
# Check git status
!cd drive/MyDrive/CS598/Final\ Project/cs598-final-project && git status

Refresh index: 100% (5/5), done.
On branch main
Your branch is up to date with 'origin/main'.

Changes not staged for commit:
  (use "git add <file>..." to update what will be committed)
  (use "git restore <file>..." to discard changes in working directory)
	modified:   intraoperative_hypotension.ipynb

no changes added to commit (use "git add" and/or "git commit -a")


In [12]:
# Copy files from git directory to working directory
!cp -r drive/MyDrive/CS598/Final\ Project/cs598-final-project/src/* .

In [11]:
# Copy files back to git directory from working directory
!cp ./*.py drive/MyDrive/CS598/Final\ Project/cs598-final-project/src/
!cd drive/MyDrive/CS598/Final\ Project/cs598-final-project && git status

On branch main
Your branch is up to date with 'origin/main'.

Changes not staged for commit:
  (use "git add <file>..." to update what will be committed)
  (use "git restore <file>..." to discard changes in working directory)
	modified:   intraoperative_hypotension.ipynb
	modified:   src/model.py

no changes added to commit (use "git add" and/or "git commit -a")


Link to paper: https://journals.plos.org/plosone/article?id=10.1371/journal.pone.0272055

In [16]:
# Imports
from model import create_1d_residual, create_flatten_and_linear


ImportError: cannot import name 'create_flatten_and_linear' from 'model' (/content/model.py)

In [9]:
# Create model building blocks
test_residual = create_1d_residual(12,3)
print(test_residual)

hi
ResidualBlock(
  (bn1): BatchNorm1d(12, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (act1): ReLU()
  (do): Dropout(p=0.5, inplace=False)
  (conv1): Conv1d(12, 12, kernel_size=(3,), stride=(1,))
  (bn2): BatchNorm1d(12, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (act2): ReLU()
  (conv2): Conv1d(12, 12, kernel_size=(3,), stride=(1,))
)
